In [4]:
import tkinter as tk
from tkinter import messagebox, simpledialog
import json
import datetime

# Singleton Pattern
class SingletonMeta(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]

# Observer Pattern
class Observer:
    def update(self, message):
        print(message)

class Logger(Observer):
    def update(self, message):
        print("Log Entry:", message)

# Factory Method Pattern
class TransactionFactory:
    @staticmethod
    def create_transaction(type, amount, category, date):
        return {"type": type, "amount": amount, "category": category, "date": date}

# Strategy Pattern
class SummaryStrategy:
    def summarize(self, transactions):
        raise NotImplementedError

class SummaryByType(SummaryStrategy):
    def summarize(self, transactions):
        summary = {}
        for t in transactions:
            summary[t['type']] = summary.get(t['type'], 0) + t['amount']
        return summary

class NetIncomeSummary(SummaryStrategy):
    def summarize(self, transactions):
        net_income = 0
        for t in transactions:
            if t['type'] == 'income':
                net_income += t['amount']
            else:
                net_income -= t['amount']
        return net_income

# Decorator Pattern
class TransactionDisplay:
    def __init__(self, transaction):
        self.transaction = transaction

    def display(self):
        return f"Transaction: {self.transaction}"

class EnhancedDisplay(TransactionDisplay):
    def display(self):
        original_display = super().display()
        return f"{original_display} [Enhanced Features]"

# Command Pattern
class AddTransactionCommand:
    def __init__(self, finance_tracker, transaction):
        self.finance_tracker = finance_tracker
        self.transaction = transaction

    def execute(self):
        self.finance_tracker.add_transaction(self.transaction)



# Adapter Pattern
class ExternalAPI:
    def get_data(self):
        return {"amount": 100, "category": "Misc", "type": "income", "date": "2024-01-10"}

class ExternalAPITransactionAdapter:
    def __init__(self, external_api):
        self.external_api = external_api

    def get_transaction(self):
        data = self.external_api.get_data()
        return TransactionFactory.create_transaction(data['type'], data['amount'], data['category'], data['date'])

# Main Finance Tracker Class using Singleton
class FinanceTracker(metaclass=SingletonMeta):
    def __init__(self):
        self.observers = []
        self.transactions = []

    def notify_observers(self, message):
        for observer in self.observers:
            observer.update(message)

    def add_observer(self, observer):
        self.observers.append(observer)

    def add_transaction(self, transaction):
        self.transactions.append(transaction)
        self.notify_observers(f"New transaction added: {transaction}")

In [5]:
import tkinter as tk
from tkinter import messagebox, Label, Entry, StringVar, Frame, Button
import pandas as pd

class AppGUI:
    def __init__(self, master):
        self.master = master
        master.title("Finance Tracker")

        self.setup_widgets()

        self.tracker = FinanceTracker()
        self.tracker.add_observer(Logger())
        self.tracker.transactions = self.load_transactions()

    def setup_widgets(self):
        frame = Frame(self.master)
        frame.pack(padx=10, pady=10)

        Label(frame, text="Transaction Type (income/expense):").grid(row=0, column=0)
        self.type_var = StringVar()
        Entry(frame, textvariable=self.type_var).grid(row=0, column=1)

        Label(frame, text="Amount:").grid(row=1, column=0)
        self.amount_var = StringVar()
        Entry(frame, textvariable=self.amount_var).grid(row=1, column=1)

        Label(frame, text="Category:").grid(row=2, column=0)
        self.category_var = StringVar()
        Entry(frame, textvariable=self.category_var).grid(row=2, column=1)

        Label(frame, text="Date (YYYY-MM-DD):").grid(row=3, column=0)
        self.date_var = StringVar()
        Entry(frame, textvariable=self.date_var).grid(row=3, column=1)

        Button(frame, text="Add Transaction", command=self.add_transaction).grid(row=4, column=0)
        Button(frame, text="View Summary by Type", command=self.view_summary_by_type).grid(row=4, column=1)
        Button(frame, text="View Net Income", command=self.view_net_income).grid(row=5, column=0)
       
        Button(frame, text="Exit", command=self.exit_app).grid(row=6, column=0, columnspan=2)

    def add_transaction(self):
        try:
            type = self.type_var.get()
            amount_str = self.amount_var.get()
            category = self.category_var.get()
            date = self.date_var.get()

            if not amount_str.strip():
                raise ValueError("Amount field is required and cannot be empty.")

            amount = float(amount_str)

            if not all([type, category, date]):
                raise ValueError("All fields must be filled out.")

            transaction = TransactionFactory.create_transaction(type, amount, category, date)
            add_command = AddTransactionCommand(self.tracker, transaction)
            add_command.execute()

            self.save_transactions(self.tracker.transactions)
            messagebox.showinfo("Success", "Transaction added successfully!")
        except ValueError as e:
            messagebox.showerror("Error", "Invalid input: " + str(e))
        except Exception as e:
            messagebox.showerror("Error", "An error occurred: " + str(e))

    
    def save_transactions(self, transactions):
        df = pd.DataFrame(transactions)
        df.to_excel('C:\\Users\\Lenovo Laptop\\OneDrive\\Desktop\\9874\\csv.xlsx', index=False)

    def load_transactions(self):
        try:
            df = pd.read_excel('C:\\Users\\Lenovo Laptop\\OneDrive\\Desktop\\9874\\csv.xlsx')
            return df.to_dict('records')
        except FileNotFoundError:
            print("No previous transactions found.")
            return []

    def view_summary_by_type(self):
        transactions = self.load_transactions()
        summary = {}
        for t in transactions:
            summary[t['type']] = summary.get(t['type'], 0) + t['amount']
        summary_text = "\n".join(f"{k}: ${v}" for k, v in summary.items())
        messagebox.showinfo("Summary by Type", summary_text)

    def view_net_income(self):
        transactions = self.load_transactions()
        net_income = sum(t['amount'] if t['type'] == 'income' else -t['amount'] for t in transactions)
        messagebox.showinfo("Net Income", f"Net Income: ${net_income}")

    def exit_app(self):
        self.master.quit()

def run_gui():
    root = tk.Tk()
    app = AppGUI(root)
    root.mainloop()

if __name__ == "__main__":
    run_gui()


Log Entry: New transaction added: {'type': 'income', 'amount': 600.0, 'category': 'A', 'date': '2024-06-20'}
Log Entry: New transaction added: {'type': 'expense', 'amount': 500.0, 'category': 'A', 'date': '2024-06-20'}


In [11]:
pip install pandas openpyxl


   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ------ -------------------------------- 41.0/250.9 kB 991.0 kB/s eta 0:00:01
   ----------------------------- ---------- 184.3/250.9 kB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 250.9/250.9 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
